## Aula 8 - Parte 3 - Recuperar comentários

Iremos recuperar todos os comentários dos 50 vídeos que recuperamos anteriormente.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('metricas_videos.csv', sep=';', encoding='utf-8')

In [ ]:
df

Acesse: https://console.developers.google.com/apis/credential e recupere a chave de acesso.

In [ ]:
from apiclient.discovery import build

In [ ]:
nome_servico = "youtube"
versao = "v3"
chave = ""

Cria o objeto com a API do Youtube, versão específica e a chave gerada.

In [ ]:
api = build(nome_servico, versao, developerKey=chave)

O método utilizado para recuperar os comentários é ```commentThreads().list()```. Alguns dos parâmetros que podem ser utilizados:

_Parâmetros obrigatórios_
- ```part``` especifica uma lista separada por vírgulas de uma ou mais propriedades de recurso ```commentThread``` que serão incluídas pela resposta da API. Os valores que podem ser incluídos são: ```id``` e ```snippet```.

_Parâmetros opcionais_
- ```videoId``` : necessário para identificar de qual vídeo os comentários devem ser recuperados.
- ```maxResults```: quantidade máxima de comentários por requisição. Máximo de 100. **Valor padrão 20.**
- ```pageToken```: token utilizado para recuperar mais de 100 comentários.



In [ ]:
def recuperar_comentarios(api, videoId, quantidade=100, token=None, part="id,snippet"):
    dados = api.commentThreads().list(
        videoId=videoId, 
        maxResults=quantidade, 
        pageToken=token,
        part=part
    ).execute()
    comentarios = [item for item in dados['items']]
    try:
        nexttoken = dados['nextPageToken']
        return (nexttoken, comentarios)
    except Exception as e:
        # Caso dê algum erro, definimos uma string de parada.
        nexttoken = "ultimo_comentario"
        return (nexttoken, comentarios)

Estrutura para salvar os dados recuperados.

In [ ]:
comentarios_dict = {
    'id_comentario' : [],
    'id_video' : [],
    'textDisplay' : [],
    'likeCount' : [],
    'authorDisplayName' : [],
    'publishedAt' : [],
    'updatedAt' : []
}

Função para estruturar os comentários recuperados.

In [ ]:
def estruturar_comentarios(api, videoId, token=None):
    novos_comentarios = recuperar_comentarios(api, videoId, token=token)
    token = novos_comentarios[0]
    comentarios = novos_comentarios[1]
    print(len(comentarios), end=" ")
    for comm in comentarios:
        comentarios_dict['id_comentario'].append(comm['id'])
        comentarios_dict['id_video'].append(comm['snippet']['topLevelComment']['snippet']['videoId'])
        comentarios_dict['textDisplay'].append(comm['snippet']['topLevelComment']['snippet']['textDisplay'])
        comentarios_dict['likeCount'].append(comm['snippet']['topLevelComment']['snippet']['likeCount'])
        comentarios_dict['authorDisplayName'].append(comm['snippet']['topLevelComment']['snippet']['authorDisplayName'])
        comentarios_dict['publishedAt'].append(comm['snippet']['topLevelComment']['snippet']['publishedAt'])
        comentarios_dict['updatedAt'].append(comm['snippet']['topLevelComment']['snippet']['updatedAt'])
        
    return token

### Recuperar todos comentários sobre todos videos

In [ ]:
from time import time, sleep

In [ ]:
import json

Para recuperar todos os comentários disponíveis dos 50 vídeos irá demorar cerca de 20 minutos.

In [ ]:
# Tempo inicial para recuperar todos os comentários dos 50 videos.
ti = time()

# Para cada item (linha) do DF
for i in range(len(df)):
    # Tempo inicial para um vídeo
    tii = time()
    
    # Recupera o Id
    print(i, '-->', df.videoId[i])
    videoId = df.videoId[i]
    
    # Tenta recuperar os comentários
    try:
        token = estruturar_comentarios(api, videoId)
        while token != "ultimo_comentario":
            try:
                token = estruturar_comentarios(api, videoId, token=token)
            except:
                # Caso dê alguma exceção, antes de tentar novamente, esperar 5 segundos.
                sleep(5)
                token = estruturar_comentarios(api, videoId, token=token)
        tff = time() - tii
        print("\nComentários recuperados para VideoId: {}. Demorou {} segundos.".format(videoId, tff))
    
    # Caso não seja possível, continue a execução para o próximo vídeo.
    except Exception as e:
        aux = json.loads(e.content)
        if aux['error']['errors'][0]['reason'] == 'commentsDisabled':
            pass
tf = time() - ti
print("\nDemorou {} segundos para recuperar os videos".format(tf))

In [ ]:
len(comentarios_dict['id_comentario'])

In [ ]:
todos_comentarios = pd.DataFrame(comentarios_dict)

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
todos_comentarios[33:38]

In [ ]:
todos_comentarios.to_csv('todos_comentarios.csv', header=True, sep=';', index=False, encoding='utf-8')